<a href="https://colab.research.google.com/github/lsteffenel/hpc-python/blob/master/dask/06_futures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/lsteffenel/hpc-python/blob/master/dask/images/dask_horizontal.svg?raw=1" align="right" width="30%">

# Utilisation de Futures avec le mode Distributed

In [ ]:
!python -m pip install dask[complete]

## Distributed futures

In [ ]:
from dask.distributed import Client
c = Client(n_workers=4)
c.cluster

Dans le chapitre précédent, nous avons montré que l'exécution d'un calcul (créé avec `delayed`) avec l'exécuteur distribué est identique à tout autre exécuteur. Cependant, nous avons maintenant accès à une fonctionnalité supplémentaire et au contrôle sur les données maintenues en mémoire.

Pour commencer, l'interface `futures` (dérivée de `concurrent.futures` intégré) permet une fonctionnalité de type map-reduce. Nous pouvons soumettre des fonctions individuelles pour évaluation avec un ensemble d'entrées, ou les évaluer sur une séquence d'entrées avec `submit()` et `map()`. Notez que l'appel retourne immédiatement, fournissant un ou plusieurs *futures*, dont le statut commence en "pending" puis devient "finished". Il n'y a pas de blocage de la session Python locale.


Voici le plus simple exemple de `submit` en action :


In [ ]:
def inc(x):
    return x + 1

fut = c.submit(inc, 1)
fut

Nous pouvons ré-exécuter la cellule suivante aussi souvent que nous le souhaitons pour sonder le statut du future. Cela pourrait bien sûr être fait dans une boucle, en pausant brièvement à chaque itération. Nous pourrions continuer notre travail, ou visualiser une barre de progression du travail en cours, ou forcer l'attente jusqu'à ce que le future soit prêt.

Si vous consultez le dashboard, le tableau de bord `status` a gagné un nouvel élément dans le flux de tâches, indiquant que `inc()` s'est terminé, et la section de progression pour le problème montre une tâche terminée et maintenue en mémoire.


In [ ]:
fut

Possibles alternatives que vous pourriez explorer :
```python
from dask.distributed import wait, progress
progress(fut)
```
afficherait une barre de progression dans ce notebook, plutôt que d’avoir à aller sur le tableau de bord. Cette barre de progression est également asynchrone et ne bloque pas l’exécution des autres morceaux de code entre‑temps.

```python
wait(fut)
```
bloquerait et forcerait le notebook à attendre jusqu’à ce que le calcul référencé par fut soit terminé. Cependant, notez que le résultat de `inc()` réside déjà dans le cluster ; exécuter le calcul maintenant ne prendrait **aucun temps**, car Dask remarque que l’on demande le résultat d’un calcul dont il a déjà connaissance. On y reviendra plus tard.

In [ ]:
# grab the information back - this blocks if fut is not ready
c.gather(fut)
# equivalent action when only considering a single future
# fut.result()

Ici, nous voyons une autre façon d'exécuter des tâches sur le cluster : lorsque vous soumettez ou mappez avec les entrées sous forme de *futures*, le *calcul se déplace vers les données* plutôt que l'inverse, et le client, dans la session Python locale, n'a jamais besoin de voir les valeurs intermédiaires. Cela est similaire à la construction du graphe avec `delayed`, et en effet, `delayed` peut être utilisé en combinaison avec les *futures*. Ici, nous utilisons l'objet `delayed` `total` d'avant.


In [ ]:
# Some trivial work that takes time
# repeated from the Distributed chapter.

from dask import delayed
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)

In [ ]:
# notice the difference from total.compute()
# notice that this cell completes immediately
fut = c.compute(total)
fut

In [ ]:
c.gather(fut) # waits until result is ready

### `Client.submit`

`submit` prend une fonction et des arguments, les pousse vers le cluster, et retourne un *Future* représentant le résultat à calculer. La fonction est passée à un processus worker pour évaluation. Notez que cette cellule retourne immédiatement, alors que le calcul peut encore être en cours sur le cluster.


In [ ]:
fut = c.submit(inc, 1)
fut

Cela ressemble beaucoup à faire un `compute()`, ci-dessus, sauf que maintenant nous passons directement la fonction et les arguments au cluster. Pour quiconque habitué à `concurrent.futures`, cela paraîtra familier. Ce nouveau `fut` se comporte de la même manière que celui d'au-dessus. Notez que nous avons maintenant écrasé la définition précédente de `fut`, qui sera collectée par le ramasse-miettes, et, par conséquent, ce résultat précédent est libéré par le cluster.

### Exercice : Reconstruire le calcul *delayed* ci-dessus en utilisant `Client.submit` à la place

Les arguments passés à `submit` peuvent être des *futures* d'autres opérations `submit` ou des objets *delayed*. Le premier cas en particulier démontre le concept de *déplacer le calcul vers les données*, qui est l'un des éléments les plus puissants de la programmation avec Dask.


In [ ]:
# Your code here

In [ ]:
x = c.submit(inc, 1)
y = c.submit(dec, 2)
total = c.submit(add, x, y)

print(total)     # This is still a future
c.gather(total)   # This blocks until the computation has finished


Chaque *future* représente un résultat détenu, ou en cours d'évaluation par le cluster. Ainsi, nous pouvons contrôler le cache des valeurs intermédiaires - lorsqu'un *future* n'est plus référencé, sa valeur est oubliée. Dans la solution, ci-dessus, des *futures* sont conservés pour chacun des appels de fonction. Ces résultats n'auraient pas besoin d'être réévalués si nous choisissions de soumettre plus de travail les nécessitant.

Nous pouvons explicitement passer des données de notre session locale vers le cluster en utilisant `scatter()`, mais il est généralement préférable de construire des fonctions qui chargent les données directement dans les *workers* eux-mêmes, afin qu'il n'y ait pas besoin de sérialiser et communiquer les données. La plupart des fonctions de chargement dans Dask, telles que `dd.read_csv`, fonctionnent de cette manière. De même, nous ne voulons normalement pas `gather()` des résultats trop volumineux en mémoire.

L'[API complète](http://distributed.readthedocs.io/en/latest/api.html) du planificateur distribué donne des détails sur l'interaction avec le cluster, qui, rappelez-vous, peut être sur votre machine locale ou éventuellement sur une ressource de calcul massive.


L'API *futures* offre un style de soumission de travail qui peut facilement émuler le paradigme *map/reduce* (voir `c.map()`) qui peut être familier à beaucoup de personnes. Les résultats intermédiaires, représentés par des *futures*, peuvent être passés à de nouvelles tâches sans avoir à les récupérer localement depuis le cluster, et du nouveau travail peut être assigné pour travailler sur la sortie de jobs précédents qui n'ont même pas encore commencé.

Généralement, toute opération Dask qui est exécutée avec `.compute()` peut être soumise pour une exécution asynchrone en utilisant `c.compute()` à la place, et cela s'applique à toutes les collections. Voici un exemple avec le calcul vu précédemment dans le chapitre Bag. Nous avons remplacé la méthode `.compute()` par la version client distribué, donc, encore une fois, nous pourrions continuer à soumettre plus de travail (peut-être basé sur le résultat du calcul), ou, dans la cellule suivante, suivre la progression du calcul. Une barre de progression similaire apparaît dans la page d'interface de surveillance.


## Calcul asynchrone
<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Rosenbrock_function.svg/450px-Rosenbrock_function.svg.png" height=200 width=200>

Un avantage de l'utilisation de l'API *futures* est que vous pouvez avoir des calculs dynamiques qui s'ajustent au fur et à mesure que les choses progressent. Ici, nous implémentons une recherche naïve simple en parcourant les résultats au fur et à mesure qu'ils arrivent, et soumettons de nouveaux points à calculer tandis que d'autres sont encore en cours d'exécution.

En observant le [tableau de bord de diagnostic](../../9002/status) pendant l'exécution, vous pouvez voir que des calculs sont exécutés concurremment tandis que d'autres sont soumis. Cette flexibilité peut être utile pour des algorithmes parallèles qui nécessitent un certain niveau de synchronisation.

Réalisons une minimisation très simple en utilisant la programmation dynamique. La fonction d'intérêt est connue sous le nom de Rosenbrock :


In [ ]:
# a simple function with interesting minima
import time

def rosenbrock(point):
    """Compute the rosenbrock function and return the point and result"""
    time.sleep(0.1)
    score = (1 - point[0])**2 + 2 * (point[1] - point[0]**2)**2
    return point, score

Configuration initiale, incluant la création d'une figure graphique. Nous utilisons Bokeh pour cela, qui permet la mise à jour dynamique de la figure au fur et à mesure que les résultats arrivent.


In [ ]:
from bokeh.io import output_notebook, push_notebook
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure, show
import numpy as np
output_notebook()

# set up plot background
N = 500
x = np.linspace(-5, 5, N)
y = np.linspace(-5, 5, N)
xx, yy = np.meshgrid(x, y)
d = (1 - xx)**2 + 2 * (yy - xx**2)**2
d = np.log(d)

p = figure(x_range=(-5, 5), y_range=(-5, 5))
p.image(image=[d], x=-5, y=-5, dw=10, dh=10, palette="Spectral11");

Nous commençons avec un point en (0, 0), et dispersons aléatoirement des points de test autour de lui. Chaque évaluation prend ~100 ms, et au fur et à mesure que les résultats arrivent, nous vérifions si nous avons un nouveau meilleur point, et choisissons des points aléatoires autour de ce nouveau meilleur point, tandis que la boîte de recherche se rétrécit.

Nous affichons la valeur de la fonction et l'emplacement du meilleur actuel à chaque fois que nous avons une nouvelle meilleure valeur.


In [ ]:
from dask.distributed import as_completed
from random import uniform

scale = 5                  # Intial random perturbation scale
best_point = (0, 0)        # Initial guess
best_score = float('inf')  # Best score so far
startx = [uniform(-scale, scale) for _ in range(10)]
starty = [uniform(-scale, scale) for _ in range(10)]

# set up plot
source = ColumnDataSource({'x': startx, 'y': starty, 'c': ['grey'] * 10})
p.circle(source=source, x='x', y='y', color='c', radius=0.1)
t = show(p, notebook_handle=True)

# initial 10 random points
futures = [c.submit(rosenbrock, (x, y)) for x, y in zip(startx, starty)]
iterator = as_completed(futures)

for res in iterator:
    # take a completed point, is it an improvement?
    point, score = res.result()
    if score < best_score:
        best_score, best_point = score, point
        print(score, point)

    x, y = best_point
    newx, newy = (x + uniform(-scale, scale), y + uniform(-scale, scale))

    # update plot
    source.stream({'x': [newx], 'y': [newy], 'c': ['grey']}, rollover=20)
    push_notebook(document=t)

    # add new point, dynamically, to work on the cluster
    new_point = c.submit(rosenbrock, (newx, newy))
    iterator.add(new_point)  # Start tracking new task as well

    # Narrow search and consider stopping
    scale *= 0.99
    if scale < 0.001:
        break
point